In [2]:
#base_dir = '/home/gbiagini/dev/nn-sero-pytorch/'
base_dir = './'

import pandas as pd
import numpy as np
from fastai import *
from fastai.tabular.all import *
#from parse import _file_handler
from tqdm import tqdm

# _file_handler()

#loci = ['A', 'B', 'C', 'DPB1', 'DQB1', 'DRB1']
loci = ["A"]
epoch = { "A":22, "B":15, "C":13, "DPB1":13, "DQB1":20, "DRB1":13 }
layer = { "A":[1500], "B":[2500], "C":[1500], "DPB1":[1500], "DQB1":[2000], "DRB1":[1500] }


for locus in loci:
  
  AAs = []
  #tng_df = pd.read_csv(base_dir + 'RSNNS_fixed/training/' + locus + '_train.csv')
  #tst_df = pd.read_csv(base_dir + 'RSNNS_fixed/testing/' + locus + '_test.csv')
  #val_df = pd.read_csv(base_dir + 'RSNNS_fixed/training/' + locus + '_validation.csv')
  tng_df = pd.read_csv(base_dir + 'old_sets/train/' + locus + '_train.csv')
  tst_df = pd.read_csv(base_dir + 'old_sets/test/' + locus + '_test.csv')
  val_df = pd.read_csv(base_dir + 'old_sets/train/' + locus + '_validation.csv')
  tng_idx = len(tng_df)
  val_idx = len(val_df) + 1
  tst_idx = len(tst_df)
  tbatch = tng_idx // 2
  if (tbatch <= 1):
    tbatch = tng_idx // 2
  vbatch = val_idx // 2
  if (vbatch <= 1):
    vbatch = val_idx // 2
  
  df = tng_df.append(val_df)

  for each in tng_df:
    x = re.search("(^\D\d*$)", each)
    if x:
      AAs.append(each)
  
  #df['serology'] = df['serology'].astype(str)

  cat_names = ['allele'] + AAs
  dep_var = [x for x in df.columns if (x not in cat_names)]
  print(len(dep_var))
  #dep_var = 'serology'
  #procs = [FillMissing, Categorify]
  procs = None
  splits = [list(range(0,tng_idx)), list(range(tng_idx,val_idx))]


  cat_names = ['allele'] + AAs
  print(len(cat_names))
  #test = TabularPandas(tst_df, cat_names=cat_names)
  
  to = (TabularPandas(df=df, procs=Categorify, cat_names=cat_names, cont_names=[], splits=splits, y_names=dep_var,y_block=MultiCategoryBlock()))
  
  dls = to.dataloaders()
  
  #acc_02 = partial(accuracy_thresh, thresh=0.99)
  #f_score = partial(fbeta, thresh=0.51)
  #f_score = FBetaMulti(0.51)
  #ra_multi = RocAucMulti

  pre_vote = {}
  avg_pred = {}
  all_models = []

  for n in range(1,2):
    #learn = tabular_learner(dls, opt_func = SGD, layers=layer[locus],
    #                          metrics=None, y_range=(0,1),
    #                          loss_func=CrossEntropyLossFlat())

    learn = tabular_learner(dls, opt_func = SGD, metrics=None,
                              loss_func=CrossEntropyLossFlat())


    lr = 0.5
    #learn.recorder.plot(suggestion=True)

    learn.fit(epoch[locus], lr)
    #learn.fit_one_cycle(epoch[locus], lr)
    #learn.fit_one_cycle(epoch[locus])
    learn.model
    #learn.recorder.plot_losses()

    test_id = list(tst_df['allele'])

    classes = data.classes
    predictions = []
    print(classes)

   
    for i in tqdm(range(0,tst_idx)):
      category = str(learn.predict(tst_df.iloc[i], thresh=0.51)[0])
      sero = category.strip('MultiCategory ')
      sero = sero.replace(';',' ')
      sero = sero.replace('a','')
      predictions.append(sero.split())
    

    pre_vote = {test_id[j]: str(predictions[j]) for j in range(len(test_id)) }
    avg_pred[str(n)] = pre_vote
    

  avg_frame = pd.DataFrame.from_dict(avg_pred)
  mode = avg_frame.mode(axis=1)
  rmode = mode[0]
  rmode = pd.DataFrame(rmode)
  rmode = rmode.reset_index()
  rmode.columns=['allele', 'serology']

  #output_preds = pd.DataFrame({'allele': test_id, 'serology': predictions})
  rmode.to_csv(base_dir + 'predictions/' + locus + '_predictions.csv', index=False)
  

1
122
Could not do one pass in your dataloader, there is something wrong in it


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [ ]:
#base_dir = '/home/gbiagini/dev/nn-sero-pytorch/'
base_dir = './'

import pandas as pd
import numpy as np
from fastai import *
from fastai.tabular.all import *
#from parse import _file_handler
from tqdm import tqdm

# _file_handler()

#loci = ['A', 'B', 'C', 'DPB1', 'DQB1', 'DRB1']
loci = ["A"]
epoch = { "A":22, "B":15, "C":13, "DPB1":13, "DQB1":20, "DRB1":13 }
layer = { "A":[1500], "B":[2500], "C":[1500], "DPB1":[1500], "DQB1":[2000], "DRB1":[1500] }


for locus in loci:
  
  AAs = []
  #tng_df = pd.read_csv(base_dir + 'RSNNS_fixed/training/' + locus + '_train.csv')
  #tst_df = pd.read_csv(base_dir + 'RSNNS_fixed/testing/' + locus + '_test.csv')
  #val_df = pd.read_csv(base_dir + 'RSNNS_fixed/training/' + locus + '_validation.csv')
  tng_df = pd.read_csv(base_dir + 'old_sets/train/' + locus + '_train.csv', low_memory=False)
  tst_df = pd.read_csv(base_dir + 'old_sets/test/' + locus + '_test.csv', low_memory=False)
  val_df = pd.read_csv(base_dir + 'old_sets/train/' + locus + '_validation.csv', low_memory=False)
  tng_idx = len(tng_df)
  val_idx = len(val_df) + 1
  tst_idx = len(tst_df)
  tbatch = tng_idx // 2
  if (tbatch <= 1):
    tbatch = tng_idx // 2
  vbatch = val_idx // 2
  if (vbatch <= 1):
    vbatch = val_idx // 2
  
  df = tng_df.append(val_df)

  for each in tng_df:
    x = re.search("(^\D\d*$)", each)
    if x:
      AAs.append(each)
  
  #df['serology'] = df['serology'].astype(str)

  cat_names = ['allele'] + AAs
  dep_var = [x for x in df.columns if (x not in cat_names)]
  print(len(dep_var))
  #dep_var = 'serology'
  procs = [FillMissing, Categorify]
  splits = [list(range(0,tng_idx)), list(range(tng_idx,val_idx))]


  cat_names = ['allele'] + AAs
  print(len(cat_names))
  #test = TabularPandas(tst_df, cat_names=cat_names)
  
  to = (TabularPandas(df=df, procs=procs, cat_names=cat_names, cont_names=[], splits=splits, y_names=dep_var,y_block=MultiCategoryBlock()))
  
  dls = to.dataloaders()
  
  #acc_02 = partial(accuracy_thresh, thresh=0.99)
  #f_score = partial(fbeta, thresh=0.51)
  #f_score = FBetaMulti(0.51)
  #ra_multi = RocAucMulti

  pre_vote = {}
  avg_pred = {}
  all_models = []

  for n in range(1,2):
    #learn = tabular_learner(dls, opt_func = SGD, layers=layer[locus],
    #                          metrics=None, y_range=(0,1),
    #                          loss_func=CrossEntropyLossFlat())

    learn = tabular_learner(dls, opt_func = SGD, metrics=None,
                              loss_func=CrossEntropyLossFlat())


    lr = 0.5
    #learn.recorder.plot(suggestion=True)

    learn.fit(epoch[locus], lr)
    #learn.fit_one_cycle(epoch[locus], lr)
    #learn.fit_one_cycle(epoch[locus])
    learn.model
    #learn.recorder.plot_losses()

    test_id = list(tst_df['allele'])

    classes = data.classes
    predictions = []
    print(classes)

   
    for i in tqdm(range(0,tst_idx)):
      category = str(learn.predict(tst_df.iloc[i], thresh=0.51)[0])
      sero = category.strip('MultiCategory ')
      sero = sero.replace(';',' ')
      sero = sero.replace('a','')
      predictions.append(sero.split())
    

    pre_vote = {test_id[j]: str(predictions[j]) for j in range(len(test_id)) }
    avg_pred[str(n)] = pre_vote
    

  avg_frame = pd.DataFrame.from_dict(avg_pred)
  mode = avg_frame.mode(axis=1)
  rmode = mode[0]
  rmode = pd.DataFrame(rmode)
  rmode = rmode.reset_index()
  rmode.columns=['allele', 'serology']

  #output_preds = pd.DataFrame({'allele': test_id, 'serology': predictions})
  rmode.to_csv(base_dir + 'predictions/' + locus + '_predictions.csv', index=False)
  

In [ ]:
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
import numpy as np
X, y1 = make_classification(n_samples=10, n_features=100, n_informative=30, n_classes=3, random_state=1)
y2 = shuffle(y1, random_state=1)
y3 = shuffle(y1, random_state=2)
Y = np.vstack((y1, y2, y3)).T
n_samples, n_features = X.shape # 10,100
n_outputs = Y.shape[1] # 3
print(Y.shape)
n_classes = 3
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
multi_target_forest.fit(X, Y).predict(X)